In [509]:
#####################
# Aleksandr Ukhatov #
#####################

# imports 
import os
import sys
import cv2
import ntpath
import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import glob
from random import randint
import skimage.io
from skimage import img_as_ubyte
import sklearn
import sklearn.manifold

from shutil import copyfile
import multiprocessing as mp
from multiprocessing import Pool

In [510]:
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

In [511]:
class_names.index("bottle")

40

In [512]:
class ResultError(Exception):
    def __init__(self, value):
        self.value = value
    def __str__(self):
        return repr(self.value)

In [513]:
def random_id():
    n = 7
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return randint(range_start, range_end)

In [514]:
class obj_class:
    def __init__(self, location, cur_obj_img_ski, cur_obj_img_cv2, frame_id, S, xdy, count):
        self.id = (1+count)*10**7 + random_id()
        self.file_address = location + str(self.id) + ".jpg"
        skimage.io.imsave(self.file_address, cur_obj_img_ski)
        self.img_cv2 = cur_obj_img_cv2
        self.img_ski = cur_obj_img_ski
        self.frames_set = {frame_id}
        self.S = S
        self.xdy = xdy
        self.kp = None
        self.des = None
        self.target_index = None
        self.target_range = None
    
    def __str__(self):
        string = str(self.file_address) + "\n" + str(self.target_index) + "\n" + str(self.target_range) + "\n"
        return string 
        
    def update(self, cur_obj_img_ski, cur_obj_img_cv2, S, xdy, frame_id, MIN_MATCH_COUNT=50):
        if match_imgs(self.img_cv2, False, cur_obj_img_cv2, False) < MIN_MATCH_COUNT: 
            return True
        if self.xdy > xdy:
            return False
#         if self.S > S:
#             return False
        
        self.S = S
        self.xdy = xdy
        self.img_ski = cur_obj_img_ski
        self.img_cv2 = cur_obj_img_cv2
        self.frames_set.add(frame_id)
        skimage.io.imsave(self.file_address, cur_obj_img_ski)

        return False
     
        
class video_class:
    def __init__(self, video_id, file_address, project_address):
        self.id = video_id
        self.file_address = file_address
        self.project_address = project_address
        self.target_index = None
        self.target_range = None
        self.target_frames = []
        self.objects_list = []
        self.target_object = None
        
    def __str__(self):
        string = str(self.file_address)
        return string 
    
class frame_class:
    def __init__(self, frame_id, video_id, file_address, segmentation):
        self.id = frame_id
        self.video_id = video_id
        self.file_address = file_address
        self.img_cv2 = cv2.imread(file_address, 0)
        self.img_ski = skimage.io.imread(file_address)
        self.segmentation = segmentation
        self.kp = None
        self.des = None
        
    def __str__(self):
        string = str(self.file_address)
        return string 
        
    def find_biggest_obj_imgs(self, num_of_biggest_objects, desired_object_id):
        
        min_y = 300
        ratio = 0.7
        
        x1 = {}
        y1 = {}
        x2 = {}
        y2 = {}
        S = {}
        xdy = {}
        biggest_obj_imgs = []
        S_of_biggest_obj_imgs = []
        xdy_of_biggest_obj_imgs = []
        

        boxes = self.segmentation['rois']        
        class_ids = self.segmentation['class_ids']
        
        useful_object_nums = []
        for i, class_id in enumerate(class_ids):
            if class_id == desired_object_id:
                useful_object_nums.append(i)

                    
        for useful_object_num in useful_object_nums:
            x1[useful_object_num], y1[useful_object_num], x2[useful_object_num], y2[useful_object_num] = boxes[useful_object_num]
            S[useful_object_num] = (y2[useful_object_num]-y1[useful_object_num])*(x2[useful_object_num]-x1[useful_object_num])
            if (x2[useful_object_num]-x1[useful_object_num])/(y2[useful_object_num]-y1[useful_object_num]) < ratio and (y2[useful_object_num]-y1[useful_object_num]) > min_y:
                xdy[useful_object_num] = (y2[useful_object_num]-y1[useful_object_num])#/(x2[useful_object_num]-x1[useful_object_num])
            else:
                xdy[useful_object_num] = 0
        r = min(num_of_biggest_objects, len(useful_object_nums))
        for i in range(r):
#             cur_mux_id = max(S, key=S.get)
            cur_mux_id = max(S, key=xdy.get)
            S_of_biggest_obj_imgs.append(S[cur_mux_id])
            xdy_of_biggest_obj_imgs.append(xdy[cur_mux_id])
            S[cur_mux_id] = 0
            xdy[cur_mux_id] = 0
            cropped = self.img_ski[x1[cur_mux_id]:x2[cur_mux_id], y1[cur_mux_id]:y2[cur_mux_id]]
            biggest_obj_imgs.append(cropped)
            
        return (biggest_obj_imgs, S_of_biggest_obj_imgs, xdy_of_biggest_obj_imgs)

In [515]:
# find matches between to given images
# inputs are image names
sift = cv2.xfeatures2d.SIFT_create()
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)

flann = cv2.FlannBasedMatcher(index_params, search_params)

def match_imgs(img1, is_obj1, img2, is_obj2, show=False, MIN_MATCH_COUNT=50):

    # read images 
#     img1 = cv2.imread(img1_name,0) # queryImage
#     img2 = cv2.imread(img2_name,0) # trainImage

    # Initiate SIFT detector
    # sift = cv2.SIFT()

    # find the keypoints and descriptors with SIFT
    
    kp1 = 0
    des1 = 0
    kp2 = 0
    des2 = 0
    
    if is_obj1:
        kp1 = img1.kp
        des1 = img1.des
        if kp1 == None:
            kp1, des1 = sift.detectAndCompute(img1.img_cv2,None)
            img1.kp = kp1
            img1.des = des1
    else:
        kp1, des1 = sift.detectAndCompute(img1,None)
        
    if is_obj2: 
        kp2 = img2.kp
        des2 = img2.des
        if kp2 == None:
            kp2, des2 = sift.detectAndCompute(img2.img_cv2,None)
            img2.kp = kp2
            img2.des = des2
    else:
        kp2, des2 = sift.detectAndCompute(img2,None)

    matches = flann.knnMatch(des1,des2,k=2)

    # store all the good matches as per Lowe's ratio test.
    good = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            good.append(m)
    
    if show:
        if len(good)>MIN_MATCH_COUNT:
            src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
            dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

            M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
            matchesMask = mask.ravel().tolist()

            h,w = img1.shape
            pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
            dst = cv2.perspectiveTransform(pts,M)

            img2 = cv2.polylines(img2,[np.int32(dst)],True,255,3, cv2.LINE_AA)

        else:
            print("Not enough matches are found - %d/%d" % (len(good),MIN_MATCH_COUNT))

            matchesMask = None

        draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                           singlePointColor = None,
                           matchesMask = matchesMask, # draw only inliers
                           flags = 2)

        img3 = cv2.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)

        plt.imshow(img3, 'gray'),plt.show()
    
    # return number of good matches
    return len(good)

In [516]:
# get target frames, with target object/car   
def get_target_frames(project_dir, img1_name, range_connstant = 30, folder_from="frames"):
    # init constants and lists 
    MIN_MATCH_COUNT = 15
    
    
    # get name of target/query image 
#     img1_name = ntpath.dirname(project_dir[:-1]) + "/" + img1_name
    # get names of images which might become target
    img2_names = sorted(glob.glob(project_dir + folder_from + "/*.jpg"))
    images_n = len(img2_names)
    
    # use multiprocessing 
#     pool = mp.Pool(mp.cpu_count())
    # find number of matches of all images with target/query image
    img1 = cv2.imread(img1_name,0) # queryImage
    goodness = []
    for img2_name in img2_names:
        goodness.append(match_imgs(img1, False, cv2.imread(img2_name,0), False))
#     goodness = [pool.apply(match_imgs, args=(img1, cv2.imread(img2_name,0))) for img2_name in img2_names]
    print(goodness)
    
    max_goodness = max(goodness)
    if max_goodness < MIN_MATCH_COUNT:
        print("Max goodness is " + str(max_goodness))
        raise ResultError("There is no target object in the input video " + ntpath.basename(project_dir[:-1]) + ".mp4")
    
    target_index = goodness.index(max_goodness)
    target_range = [max(0, target_index-range_connstant), min(target_index+range_connstant, images_n)]
    
    return (target_index, target_range)



In [521]:
def video_to_frames(video_file, place="data/projects_noname/", folder_for_frames="frames", pr_dir=0):
    # init constants
    scale = 1
    ms = 200

    
    print(video_file)

    # init project directory
    video_name = ((ntpath.basename(video_file)).split("."))[0]
    os.system("mkdir " + place)
    os.system("mkdir " + place +  video_name)
    os.system("mkdir "+ place + video_name + "/" + folder_for_frames)
    # save current project directory name
    project_dir = os.getcwd() + "/" + place + video_name + "/" 
    if pr_dir:
        return project_dir
    
    count = 1
    # read image
    vidcap = cv2.VideoCapture(video_file)
    # read first image
    success, image = vidcap.read()
    while success:
#         print('Read a new frame #' + str(count))
        # resize image
        dim = (int(image.shape[1] * scale), int(image.shape[0] * scale))
        image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
        # save image
        cv2.imwrite(project_dir + folder_for_frames + "/" + video_name + "_frame_" + str(count).zfill(3) + ".jpg", image)     # save frame as JPEG file      
        # read new image
        vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*ms))
        success, image = vidcap.read()
        # increase count
        count += 1  
        # limitations
        if count > 100:
            break 
    print("Obtained " + str(count-1) + " images")
    # return current project directory
    return project_dir

In [518]:
# get model for MASK RCNN
def get_model():
    # Root directory of the project
    ROOT_DIR = os.getcwd()
    # Import Mask RCNN
    sys.path.append(os.path.join(ROOT_DIR, "code/"))  # To find local version of the library
    from mrcnn import utils
    import mrcnn.model as modellib
    from mrcnn import visualize
    # Import COCO config
    sys.path.append(os.path.join(ROOT_DIR, "code/coco/"))  # To find local version
    import coco
    # Directory to save logs and trained model
    MODEL_DIR = os.path.join(ROOT_DIR, "logs/")
    # Local path to trained weights file
    COCO_MODEL_PATH = os.path.join(ROOT_DIR, "MASK_RCNN_models/mask_rcnn_coco.h5")
    # Download COCO trained weights from Releases if needed
    if not os.path.exists(COCO_MODEL_PATH):
        raise ResultError("Can not find COCO_MODELL")
    #     utils.download_trained_weights(COCO_MODEL_PATH)

    # Directory of images to run detection on
    class InferenceConfig(coco.CocoConfig):
        # Set batch size to 1 since we'll be running inference on
        # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
        GPU_COUNT = 1
        IMAGES_PER_GPU = 1

    config = InferenceConfig()
    # config.display()

    # Create model object in inference mode.
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)
    # Load weights trained on MS-COCO
    model.load_weights(COCO_MODEL_PATH, by_name=True)

    # COCO Class names
    # Index of the class in the list is its ID. For example, to get ID of
    # the teddy bear class, use: class_names.index('teddy bear')
    
    return model

In [519]:
# Object Detection  
# Label points on images 
def label_points(IMAGE_DIR, target_range, model):
 
    # init dictinary for labeled points
    labeled_points = []
    image_files = [] 
    # for each image apply MASK RCNN
    lst = sorted(glob.glob(IMAGE_DIR + "*.jpg"))
    for image_id, image_file in enumerate(lst):
        if image_id < target_range[0] or image_id >= target_range[1]:
            continue
        print(image_file)
        # read image
        image = skimage.io.imread(image_file)

        # Run detection
        labeled_points.append((model.detect([image], verbose=1))[0])
        image_files.append(image_file)
        # visualization of MASKS
        
#         from mrcnn import visualize
#         results = labeled_points[-1]
#         visualize.display_instances(image, results['rois'], results['masks'], results['class_ids'], class_names, results['scores'])
    return (image_files, labeled_points)


In [522]:
place = "data/projects3/"
folder_for_objects = "objects"
folder_for_frames = "frames"
ROOT_DIR = os.getcwd()


while ((list(filter(bool, ROOT_DIR.split("/"))))[-1] != "distances"):
    os.chdir('..')
    ROOT_DIR = os.getcwd()
    

# get model
model = get_model()
videos = []

video_files = []
video_files.append("/Users/ukhatov/Documents/Projects/distances/data/videos/20191211_132104.mp4")
labeled_points = 0
image_files = 0
for video_id, video_file in enumerate(video_files):
    project_dir = video_to_frames(video_file, place, folder_for_frames)
    video_i = video_class(video_id, video_file, project_dir)
    videos.append(video_i)
    
    print("get target frames")
    # get target frames
    target_index, target_range = get_target_frames(project_dir, "/Users/ukhatov/Documents/Projects/distances/data/target2.jpg", folder_from=folder_for_frames)
    video_i.target_index = target_index
    video_i.target_range = target_range
    
    print("label points")
    # label points
    image_files, labeled_points = label_points(video_i.project_address + folder_for_frames + "/", target_range, model)
    print("get frames")
    


/Users/ukhatov/Documents/Projects/distances/data/videos/20191211_132104.mp4
Obtained 64 images
get target frames
[21, 15, 17, 18, 18, 18, 20, 20, 16, 14, 23, 20, 18, 22, 32, 27, 28, 27, 28, 28, 29, 42, 42, 56, 57, 83, 127, 134, 193, 252, 493, 149, 84, 55, 53, 28, 16, 23, 12, 15, 13, 18, 12, 16, 24, 26, 27, 27, 15, 17, 17, 30, 27, 21, 18, 14, 17, 18, 18, 10, 11, 11, 12, 14]
label points
/Users/ukhatov/Documents/Projects/distances/data/projects3/20191211_132104/frames/20191211_132104_frame_001.jpg
Processing 1 images
image                    shape: (1080, 1920, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1920.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
/Users/ukhatov/Documents/Projects/distances/data/projects3/20191211_132104/frames/20191211_132104

molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1920.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
/Users/ukhatov/Documents/Projects/distances/data/projects3/20191211_132104/frames/20191211_132104_frame_017.jpg
Processing 1 images
image                    shape: (1080, 1920, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1920.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
/Users/ukhatov/Documents/Projects/distances/data/projects3/20191211_132104/frames/20191211_132104_frame_018.jpg
Processing 1 images
image                    shape: (1080, 1920, 3)       min:    0.00

molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  148.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1920.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
/Users/ukhatov/Documents/Projects/distances/data/projects3/20191211_132104/frames/20191211_132104_frame_033.jpg
Processing 1 images
image                    shape: (1080, 1920, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  148.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1920.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
/Users/ukhatov/Documents/Projects/distances/data/projects3/20191211_132104/frames/20191211_132104_frame_034.jpg
Processing 1 images
image                    shape: (1080, 1920, 3)       min:    0.00

molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  149.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1920.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
/Users/ukhatov/Documents/Projects/distances/data/projects3/20191211_132104/frames/20191211_132104_frame_049.jpg
Processing 1 images
image                    shape: (1080, 1920, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  149.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1920.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
/Users/ukhatov/Documents/Projects/distances/data/projects3/20191211_132104/frames/20191211_132104_frame_050.jpg
Processing 1 images
image                    shape: (1080, 1920, 3)       min:    0.00

In [523]:
videos[0].target_range
videos[0].target_index

30

In [524]:
global_a = videos[0].target_range
global_b = videos[0].target_index

In [525]:
place = "data/projects3/"
folder_for_objects = "objects"
folder_for_frames = "frames"
ROOT_DIR = os.getcwd()


while ((list(filter(bool, ROOT_DIR.split("/"))))[-1] != "distances"):
    os.chdir('..')
    ROOT_DIR = os.getcwd()
    

# get model
# model = get_model()
videos = []

video_files = []
video_files.append("/Users/ukhatov/Documents/Projects/distances/data/videos/20191211_132104.mp4")
project_dir = 0
for video_id, video_file in enumerate(video_files):
    project_dir = video_to_frames(video_file, place, folder_for_frames)
    video_i = video_class(video_id, video_file, project_dir)
    videos.append(video_i)
    
    print("get target frames")
    # get target frames
#     target_index, target_range = get_target_frames(project_dir, "target5.jpg", folder_from=folder_for_frames)
    video_i.target_index = global_b
    video_i.target_range = global_a
    
#     print("label points")
    # label points
#     image_files, labeled_points = label_points(video_i.project_address + folder_for_frames + "/", target_range, model)

# for video_i in videos:
    for frame_id, segmentation in enumerate(labeled_points):
        frame_i = frame_class(frame_id, video_id, image_files[frame_id], segmentation)
        video_i.target_frames.append(frame_i)
    
#     print("Information from video #" + str(video_id) + " is obtained")

/Users/ukhatov/Documents/Projects/distances/data/videos/20191211_132104.mp4
Obtained 64 images
get target frames


In [526]:
# desired_object_id = 48 # apple
desired_object_id = 3 # car
# desired_object_id = 74 # book
# desired_object_id = 40 # bottle
MIN_MATCH_COUNT = 50

count_objects = 0
num_of_biggest_obj_imgs = 3
for video in videos:
    video.objects_list = []
    os.system("mkdir "+ video.project_address + folder_for_objects)
    for frame in video.target_frames:
        print(frame.id)
        biggest_obj_imgs, S_of_biggest_obj_imgs, xdy_of_biggest_obj_imgs = frame.find_biggest_obj_imgs(num_of_biggest_obj_imgs, desired_object_id) 
        flag = True
        for i, cur_obj_img_ski in enumerate(biggest_obj_imgs):
            cur_obj_img_cv2 = img_as_ubyte(cur_obj_img_ski)
            if xdy_of_biggest_obj_imgs[i] == 0:
                continue
            for obj in video.objects_list:
                flag = flag and (obj.update(cur_obj_img_ski, cur_obj_img_cv2, S_of_biggest_obj_imgs[i], xdy_of_biggest_obj_imgs[i], frame.id, MIN_MATCH_COUNT)) 
            if flag:
                cur_obj = obj_class(video.project_address + folder_for_objects + "/", cur_obj_img_ski, cur_obj_img_cv2, frame_id, S_of_biggest_obj_imgs[i], xdy_of_biggest_obj_imgs[i], count_objects)
                count_objects += 1
                video.objects_list.append(cur_obj)
            

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


In [ ]:
print(videos[0].project_address + folder_for_frames)
print(videos[0].target_range)
print(len(labeled_points))
print(image_files)

In [ ]:
file_address = "/Users/ukhatov/Documents/Projects/distances/data/projects3/20191005_145816/objects/aaaaa.jpg"
print(file_address)
skimage.io.imsave(file_address, videos[0].target_frames[0].img_ski[185:308, 598:725])

In [527]:
for video in videos:
    for obj in video.objects_list:
        print(obj.id)

19580587
28723755
37412552
44380234
59896712
66275706


In [528]:
for video in videos:
    for obj in video.objects_list:
        print(obj.file_address)
        print(project_dir)
        obj.target_index, obj.target_range = get_target_frames(project_dir, obj.file_address, range_connstant=15)

#         for frame in video.target_frames:
            
#             print(frame.id)
#             if match_imgs(obj, True, frame, True) > MIN_MATCH_COUNT:
#                 obj.frames_set.add(frame.id)

/Users/ukhatov/Documents/Projects/distances/data/projects3/20191211_132104/objects/19580587.jpg
/Users/ukhatov/Documents/Projects/distances/data/projects3/20191211_132104/
[238, 270, 451, 1259, 374, 226, 115, 85, 63, 46, 34, 27, 18, 12, 12, 9, 20, 12, 10, 11, 7, 4, 10, 12, 8, 15, 13, 11, 12, 13, 11, 7, 15, 8, 21, 14, 17, 4, 8, 17, 11, 19, 6, 13, 11, 11, 19, 12, 18, 8, 15, 13, 13, 10, 18, 9, 11, 9, 9, 10, 19, 8, 14, 14]
/Users/ukhatov/Documents/Projects/distances/data/projects3/20191211_132104/objects/28723755.jpg
/Users/ukhatov/Documents/Projects/distances/data/projects3/20191211_132104/
[20, 7, 9, 17, 17, 15, 19, 22, 17, 35, 64, 100, 156, 605, 164, 84, 29, 7, 9, 7, 5, 5, 5, 3, 4, 5, 6, 4, 3, 5, 6, 2, 4, 7, 6, 3, 7, 9, 9, 7, 6, 5, 7, 12, 2, 9, 6, 6, 6, 5, 7, 10, 9, 8, 7, 10, 10, 6, 6, 4, 3, 7, 5, 7]
/Users/ukhatov/Documents/Projects/distances/data/projects3/20191211_132104/objects/37412552.jpg
/Users/ukhatov/Documents/Projects/distances/data/projects3/20191211_132104/
[5, 2, 4, 7, 5, 6

In [560]:

for video in videos:
    for obj in video.objects_list:
        obj.target_range = [obj.target_index - 15, obj.target_index + 15]
        print(obj.target_range)
        print(obj.target_index)
    video.objects_list = sorted(video.objects_list, key=lambda x: x.target_index, reverse=False)

[-12, 18]
3
[-2, 28]
13
[5, 35]
20
[15, 45]
30
[31, 61]
46
[39, 69]
54


In [508]:
# 9
# 17
# 26
# 35
# 47
import copy
import random
for video in videos:
    a = copy.deepcopy(video.objects_list)
#     video.objects_list[0].target_index = 9
    random.shuffle(a)
    for obj in video.objects_list:
        print(obj.target_index)
    print("\n")
    for obj in a:
        print(obj.target_index)
    print("\n")
    b = sorted(a, key=lambda x: x.target_index, reverse=False)
    for obj in b:
        print(obj.target_index)

9
17
26
35
47


9
26
47
17
35


9
17
26
35
47


In [567]:
co_frequency = {}
co_distance = {}
def find_co_frequency(range1, range2):
    if range1[0] > range2[0] or range1[1] > range2[1]:
        cur_range1 = range1
        range1 = range2
        range2 = cur_range1
    if range1[1] < range2[0]:
        return 0
    return (range1[1] - range2[0])
    
def dist_from_freq(f):
    if f == 0:
        return -1
    a = 2860.0
    b = 50.0
    c = -34.1
    return a/(f+b)+c

for video in videos:
    for i, obj_i in enumerate(video.objects_list):
        co_frequency[i] = {}
        co_distance[i] = {}
        for j, obj_j in enumerate(video.objects_list):
            if i == j:
                co_frequency[i][j] = float('inf')
                co_distance[i][j] = 0
                continue
            co_frequency[i][j] = find_co_frequency(obj_i.target_range, obj_j.target_range)
            co_distance[i][j] = dist_from_freq(co_frequency[i][j])

In [568]:
for i in co_distance:
    print(i, co_distance[i])
print("\n")
for i in co_distance:
    for j in co_distance[i]:
        if co_distance[i][j] == -1:
            if i >= j:
                continue
            co_distance[i][j] = 0
            for k in range(j - i):
                co_distance[i][j] += co_distance[i+k][i+k+1]
            co_distance[j][i] = co_distance[i][j]
    print(i, co_distance[i])

0 {0: 0, 1: 6.757142857142853, 2: 11.296825396825398, 3: 19.862264150943396, 4: -1, 5: -1}
1 {0: 6.757142857142853, 1: 0, 2: 5.0780821917808225, 3: 11.296825396825398, 4: -1, 5: -1}
2 {0: 11.296825396825398, 1: 5.0780821917808225, 2: 0, 3: 6.757142857142853, 4: 18.86296296296296, 5: -1}
3 {0: 19.862264150943396, 1: 11.296825396825398, 2: 6.757142857142853, 3: 0, 4: 10.587499999999999, 5: 16.971428571428568}
4 {0: -1, 1: -1, 2: 18.86296296296296, 3: 10.587499999999999, 4: 0, 5: 5.62222222222222}
5 {0: -1, 1: -1, 2: -1, 3: 16.971428571428568, 4: 5.62222222222222, 5: 0}


0 {0: 0, 1: 6.757142857142853, 2: 11.296825396825398, 3: 19.862264150943396, 4: 29.179867906066526, 5: 34.80209012828875}
1 {0: 6.757142857142853, 1: 0, 2: 5.0780821917808225, 3: 11.296825396825398, 4: 22.422725048923674, 5: 28.044947271145894}
2 {0: 11.296825396825398, 1: 5.0780821917808225, 2: 0, 3: 6.757142857142853, 4: 18.86296296296296, 5: 22.96686507936507}
3 {0: 19.862264150943396, 1: 11.296825396825398, 2: 6.7571

In [569]:
distance_matrix = np.asmatrix([list(i.values()) for i in list(co_distance.values())])
print(distance_matrix)
coordinates, l = sklearn.manifold.smacof(distance_matrix, n_components=1)
# print(coordinates)
coordinates -= coordinates[3]
print(coordinates)

[[ 0.          6.75714286 11.2968254  19.86226415 29.17986791 34.80209013]
 [ 6.75714286  0.          5.07808219 11.2968254  22.42272505 28.04494727]
 [11.2968254   5.07808219  0.          6.75714286 18.86296296 22.96686508]
 [19.86226415 11.2968254   6.75714286  0.         10.5875     16.97142857]
 [29.17986791 22.42272505 18.86296296 10.5875      0.          5.62222222]
 [34.80209013 28.04494727 22.96686508 16.97142857  5.62222222  0.        ]]
[[ 18.70924905]
 [ 11.74045681]
 [  7.09489452]
 [  0.        ]
 [-10.84558831]
 [-16.34170824]]


In [572]:
np.diff(-coordinates[:, 0])

array([ 6.96879223,  4.64556229,  7.09489452, 10.84558831,  5.49611993])

In [570]:
(coordinates[:, 0])

array([ 18.70924905,  11.74045681,   7.09489452,   0.        ,
       -10.84558831, -16.34170824])

In [546]:
co_frequency

{0: {0: inf, 1: 18, 2: 13, 3: 3, 4: 0, 5: 0},
 1: {0: 18, 1: inf, 2: 23, 3: 13, 4: 0, 5: 0},
 2: {0: 13, 1: 23, 2: inf, 3: 20, 4: 4, 5: 0},
 3: {0: 3, 1: 13, 2: 20, 3: inf, 4: 14, 5: 6},
 4: {0: 0, 1: 0, 2: 4, 3: 14, 4: inf, 5: 21},
 5: {0: 0, 1: 0, 2: 0, 3: 6, 4: 21, 5: inf}}

In [547]:
videos[0].target_index

30

In [548]:
videos[0].target_range

[0, 60]

In [549]:
for frame in videos[0].target_frames:
    print(frame.file_address)

/Users/ukhatov/Documents/Projects/distances/data/projects3/20191211_132104/frames/20191211_132104_frame_001.jpg
/Users/ukhatov/Documents/Projects/distances/data/projects3/20191211_132104/frames/20191211_132104_frame_002.jpg
/Users/ukhatov/Documents/Projects/distances/data/projects3/20191211_132104/frames/20191211_132104_frame_003.jpg
/Users/ukhatov/Documents/Projects/distances/data/projects3/20191211_132104/frames/20191211_132104_frame_004.jpg
/Users/ukhatov/Documents/Projects/distances/data/projects3/20191211_132104/frames/20191211_132104_frame_005.jpg
/Users/ukhatov/Documents/Projects/distances/data/projects3/20191211_132104/frames/20191211_132104_frame_006.jpg
/Users/ukhatov/Documents/Projects/distances/data/projects3/20191211_132104/frames/20191211_132104_frame_007.jpg
/Users/ukhatov/Documents/Projects/distances/data/projects3/20191211_132104/frames/20191211_132104_frame_008.jpg
/Users/ukhatov/Documents/Projects/distances/data/projects3/20191211_132104/frames/20191211_132104_frame_

In [550]:
global_a

[0, 60]

In [551]:
global_b

30